# Pipeline NLP Helpdesk Softland – Módulo CW (GESCO)

Este notebook corresponde a la segunda entrega (dataset más robusto extraído desde SQL Server), utilizando el campo CodProd para asociar cada registro al módulo Softland.
En esta entrega se trabaja solo CW. La misma lógica se replica luego para IW, OW, SW, etc.

Objetivo: clasificar y sintetizar patrones de tickets CW para proponer productos de consultoría y capacitación.


## PASO 0 – Preparación del entorno (Google Colab)

In [ ]:

# PASO 0.1 – Instalación de dependencias (Colab)
# Nota: si ya instalaste antes, puedes omitir/re-ejecutar sin problema.

!apt-get update -y
!apt-get install -y unixodbc unixodbc-dev
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update -y
!ACCEPT_EULA=Y apt-get install -y msodbcsql18

!pip -q install pyodbc sqlalchemy pandas numpy matplotlib scikit-learn
!pip -q install sentence-transformers bertopic umap-learn hdbscan


In [ ]:

# PASO 0.2 – Imports
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyodbc
from getpass import getpass

from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer


## PASO 1 – Conexión a SQL Server y extracción (solo CW)

In [ ]:

# PASO 1.1 – Parámetros de conexión (ajusta solo si cambia el servidor)
SERVER = "sqlreplica01.softlandcloud.cl"
DATABASE = "GESCO"

# PASO 1.2 – Solicitar credenciales (no quedan guardadas)
user = input("Usuario SQL: ")
password = getpass("Contraseña SQL: ")

# PASO 1.3 – Conexión vía ODBC Driver 18
conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={user};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
    "Connection Timeout=60;"
)

cn = pyodbc.connect(conn_str, autocommit=True)
cn.timeout = 120
print("Conectado OK a", DATABASE)


In [ ]:

# PASO 1.4 – Query (solo CW)
# Nota: esta versión es intencionalmente simple para la entrega CW.
# Si quieres incluir más limpieza en SQL, puedes reemplazar esta query por tu versión filtrada.

query_cw = '''
SELECT
    d.CodHw,
    d.FchHrIni,
    d.CodRes,
    CAST(d.Descripcion AS varchar(max)) AS Descripcion,
    e.CodAux,
    e.CodProd
FROM softland.hwdeseve d
INNER JOIN softland.hweventohlp e
    ON d.CodHw = e.CodHw
    AND d.FchHrIni = e.FchHrIni
    AND d.CodRes = e.CodRes
WHERE
    d.FchHrIni >= '2024-01-01'
    AND e.CodProd = 'CW'
    AND d.Descripcion IS NOT NULL
'''

df_raw = pd.read_sql(query_cw, cn)
print("Filas extraídas:", len(df_raw))
df_raw.head()


## PASO 2 – Limpieza y normalización del texto

In [ ]:

# PASO 2.1 – Preparación base
df = df_raw.copy()
df["Descripcion"] = df["Descripcion"].astype(str)

df["len_texto"] = df["Descripcion"].str.len()

def normalizar_texto(s: str) -> str:
    s = (s or "").lower()
    s = s.replace("\r", " ").replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["texto_norm"] = df["Descripcion"].apply(normalizar_texto)
df["len_texto_norm"] = df["texto_norm"].str.len()

MIN_LEN = 60
df_work = df[df["len_texto_norm"] >= MIN_LEN].copy()

print("Filas tras filtro de longitud:", len(df_work))
df_work[["CodHw","FchHrIni","CodRes","CodProd","len_texto_norm"]].head()


In [ ]:

# PASO 2.2 – Flags simples (orientativos)
def flag_contains(text, patterns):
    t = text or ""
    return int(any(p in t for p in patterns))

df_work["flag_ia"] = df_work["texto_norm"].apply(lambda t: flag_contains(t, [" ia", "gestión de ia", "gestion de ia", "ia."]))
df_work["flag_qa"] = df_work["texto_norm"].apply(lambda t: flag_contains(t, [" qa", "quality", "calidad"]))
df_work["flag_cx"] = df_work["texto_norm"].apply(lambda t: flag_contains(t, [" cx", "cliente", "client"]))
df_work["flag_web"] = df_work["texto_norm"].apply(lambda t: flag_contains(t, ["viaweb", "vía web", "via web", "web"]))

flags_pct = {
    "flag_ia": round(df_work["flag_ia"].mean()*100, 2),
    "flag_qa": round(df_work["flag_qa"].mean()*100, 2),
    "flag_cx": round(df_work["flag_cx"].mean()*100, 2),
    "flag_web": round(df_work["flag_web"].mean()*100, 2),
}
print("Flags (%):", flags_pct)


In [ ]:

# PASO 2.3 – Campo final para modelado
def limpiar_para_modelo(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = s.replace("\r", " ").replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_work["texto_modelo"] = df_work["Descripcion"].astype(str).apply(limpiar_para_modelo)
df_work["len_texto_modelo"] = df_work["texto_modelo"].str.len()

df_work[["len_texto_modelo"]].describe()


## PASO 3 – EDA mínimo (texto y volumen)

In [ ]:

# PASO 3.1 – Resumen longitudes
print("Resumen longitudes:")
display(df_work["len_texto_modelo"].describe())

# PASO 3.2 – Histograma (sin fijar colores)
plt.figure()
df_work["len_texto_modelo"].clip(upper=df_work["len_texto_modelo"].quantile(0.99)).hist(bins=50)
plt.title("Distribución de longitudes (recortado al p99)")
plt.xlabel("Longitud texto_modelo")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:

# PASO 3.3 – Volumen mensual
df_work["anio_mes"] = pd.to_datetime(df_work["FchHrIni"]).dt.to_period("M").astype(str)
vol_mensual = df_work.groupby("anio_mes", as_index=False).size().rename(columns={"size":"tickets"})
display(vol_mensual.tail(12))

plt.figure()
plt.plot(vol_mensual["anio_mes"], vol_mensual["tickets"])
plt.title("CW – Volumen mensual de tickets")
plt.xlabel("Año-Mes")
plt.ylabel("Tickets")
plt.xticks(rotation=45)
plt.show()


## PASO 4 – Embeddings (representación semántica)

In [ ]:

# PASO 4.1 – Cargar modelo de embeddings
modelo_embeddings = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# PASO 4.2 – Corpus
corpus = df_work["texto_modelo"].astype(str).tolist()
print("Textos para embeddings:", len(corpus))


In [ ]:

# PASO 4.3 – Calcular embeddings
embeddings = modelo_embeddings.encode(
    corpus,
    show_progress_bar=True,
    batch_size=64
)
print("Shape embeddings:", embeddings.shape)


## PASO 5 – Modelamiento de tópicos (BERTopic)

In [ ]:

# PASO 5.1 – Vectorizer y BERTopic
vectorizer_model = CountVectorizer(
    ngram_range=(1,2),
    min_df=2,
    max_df=0.90
)

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    min_topic_size=200,
    verbose=True
)

topics, probs = topic_model.fit_transform(corpus, embeddings)
df_work["topico_cw"] = topics

info = topic_model.get_topic_info()
display(info.head(20))


In [ ]:

# PASO 5.2 – Gráfico distribución de tópicos (Top 15)
dist_top = df_work["topico_cw"].value_counts().head(15)

plt.figure()
dist_top.sort_values().plot(kind="barh")
plt.title("CW – Distribución de macro-tópicos (Top 15)")
plt.xlabel("Cantidad")
plt.ylabel("Tópico")
plt.show()


## PASO 6 – Muestras para validación (lectura rápida)

In [ ]:

# PASO 6.1 – Muestras por tópico (10 ejemplos)
def muestras_por_topico(df_in, col_topico, n=10, seed=42):
    out = []
    for t in sorted(df_in[col_topico].dropna().unique()):
        df_t = df_in[df_in[col_topico] == t]
        if df_t.shape[0] == 0:
            continue
        sample = df_t.sample(min(n, df_t.shape[0]), random_state=seed)
        for _, r in sample.iterrows():
            out.append({
                "topico": int(t),
                "CodHw": r.get("CodHw", None),
                "FchHrIni": r.get("FchHrIni", None),
                "CodRes": r.get("CodRes", None),
                "texto_modelo": r.get("texto_modelo", "")
            })
    return pd.DataFrame(out)

df_val = muestras_por_topico(df_work, "topico_cw", n=10)
display(df_val.head(30))


## PASO 7 – Categorías de consultoría CW (reglas editables)

In [ ]:

# PASO 7.1 – Reglas base por keywords (ajusta con tu criterio)
MAP_CATEGORIAS = [
    ("BANCOS_CONCILIACION", ["concili", "cartola", "banco", "cheque", "transfer", "deposit", "cuenta corriente"]),
    ("PERIODOS_CIERRE", ["abrir periodo", "apertura", "cierre", "reapert", "reproceso", "periodo contable"]),
    ("COMPROBANTES_ASIENTOS", ["comprobante", "asiento", "correlativo", "diario", "mayor", "folio"]),
    ("REPORTES_BALANCES", ["balance", "estado resultado", "libro mayor", "libro diario", "informe", "reporte"]),
    ("SALDOS_AUXILIARES_CUADRATURA", ["saldo", "auxiliar", "cuadratura", "movimiento", "cuenta", "diferencia"]),
    ("GESTION_ADMIN", ["se llama", "se procede", "contactar", "correo", "seguimiento", "compromiso", "oficina"]),
]

def asignar_categoria(texto: str) -> str:
    t = (texto or "").lower()
    for cat, kws in MAP_CATEGORIAS:
        for kw in kws:
            if kw in t:
                return cat
    return "OTROS"

df_work["categoria_consultoria_cw"] = df_work["texto_modelo"].apply(asignar_categoria)

MAP_PRODUCTO = {
    "BANCOS_CONCILIACION": "Curso/Asesoría: Conciliación bancaria en Softland",
    "PERIODOS_CIERRE": "Curso/Asesoría: Cierres, reaperturas y reprocesos contables",
    "COMPROBANTES_ASIENTOS": "Curso/Asesoría: Comprobantes, asientos y correlativos",
    "REPORTES_BALANCES": "Curso: Reportes, balances y libros contables",
    "SALDOS_AUXILIARES_CUADRATURA": "Servicio/Cursos: Cuadratura de cuentas, saldos y auxiliares",
    "GESTION_ADMIN": "Optimización operativa: guiones, seguimiento y atención",
    "OTROS": "Revisión caso a caso / diagnóstico",
}

df_work["producto_sugerido_cw"] = df_work["categoria_consultoria_cw"].map(MAP_PRODUCTO).fillna("Diagnóstico")

df_work[["CodHw","FchHrIni","CodProd","categoria_consultoria_cw","producto_sugerido_cw","texto_modelo"]].head()


## PASO 8 – Tabla ejecutiva final CW (categoría, volumen, porcentaje, producto)

In [ ]:

# PASO 8.1 – Tabla final
tabla_final = (
    df_work.groupby("categoria_consultoria_cw", as_index=False)
    .size()
    .rename(columns={"size": "cantidad"})
)
tabla_final["porcentaje"] = (tabla_final["cantidad"] / tabla_final["cantidad"].sum() * 100).round(2)
tabla_final["producto_sugerido"] = tabla_final["categoria_consultoria_cw"].map(MAP_PRODUCTO).fillna("Diagnóstico")
tabla_final = tabla_final.sort_values("cantidad", ascending=False)

display(tabla_final)


In [ ]:

# PASO 8.2 – Gráfico distribución por categoría
plt.figure()
tabla_final.sort_values("cantidad").plot(kind="barh", x="categoria_consultoria_cw", y="cantidad", legend=False)
plt.title("CW – Distribución por categoría de consultoría")
plt.xlabel("Cantidad")
plt.ylabel("Categoría")
plt.show()


## PASO 9 – Evolución mensual por categoría (Top 5)

In [ ]:

# PASO 9.1 – Evolución mensual (Top 5)
top5 = tabla_final["categoria_consultoria_cw"].head(5).tolist()

evo = (
    df_work[df_work["categoria_consultoria_cw"].isin(top5)]
    .groupby(["anio_mes", "categoria_consultoria_cw"], as_index=False)
    .size()
    .rename(columns={"size": "cantidad"})
)

pivot = evo.pivot(index="anio_mes", columns="categoria_consultoria_cw", values="cantidad").fillna(0)

plt.figure()
pivot.plot(kind="line")
plt.title("CW – Evolución mensual (Top 5 categorías)")
plt.xlabel("Año-Mes")
plt.ylabel("Cantidad")
plt.xticks(rotation=45)
plt.show()

display(evo.tail(20))


## PASO 10 – Dataset final clasificado y export

In [ ]:

# PASO 10.1 – Dataset final (mínimo para entrega/BI)
cols_final = [
    "CodHw","FchHrIni","CodRes","CodAux","CodProd",
    "topico_cw","categoria_consultoria_cw","producto_sugerido_cw",
    "texto_modelo"
]
df_final = df_work[cols_final].copy()

print("Filas df_final:", len(df_final))
df_final.head()


In [ ]:

# PASO 10.8 – EXPORT (OPCIONAL) A CSV PARA ENTREGA
tabla_final.to_csv("cw_tabla_estrategica.csv", index=False, encoding="utf-8-sig")
df_final.to_csv("cw_dataset_clasificado.csv", index=False, encoding="utf-8-sig")

print("Exportado: cw_tabla_estrategica.csv")
print("Exportado: cw_dataset_clasificado.csv")
